In [ ]:
#| default_exp learning_interface

# Learning Interface

> Gradio interface

## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
from typing import Dict, List, Optional, Tuple, Any, AsyncGenerator
import gradio as gr
import numpy as np
from pathlib import Path
import asyncio
from datetime import datetime
import pandas as pd
from wardbuddy.clinical_tutor import ClinicalTutor
from wardbuddy.learning_context import setup_logger, LearningCategory, SmartGoal
from wardbuddy.auth import AuthManager
import json

logger = setup_logger(__name__)

C:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'wardbuddy.auth'

## Learning Interface

In [ ]:
#| export
def create_css() -> str:
    """Create custom CSS for interface styling with auth and mobile support."""
    return """
    /* Base styles */
    .gradio-container {
        background-color: #0f172a !important;
        max-width: 100% !important;
        padding: 1rem !important;
    }
    
    /* Auth container */
    .auth-container {
        max-width: 400px !important;
        margin: 2rem auto !important;
        padding: 2rem !important;
        background-color: #1e293b !important;
        border-radius: 0.5rem !important;
        border: 1px solid #334155 !important;
    }
    
    .auth-input input {
        background-color: #1e293b !important;
        border: 1px solid #334155 !important;
        color: #f1f5f9 !important;
        font-size: 1rem !important;
        padding: 0.75rem !important;
    }
    
    .auth-button {
        width: 100% !important;
        margin-top: 1rem !important;
        background-color: #2563eb !important;
        color: white !important;
    }
    
    /* Discussion elements */
    .discussion-status {
        color: #94a3b8;
        font-size: 0.9em;
        margin-bottom: 1rem;
    }
    
    .chat-window {
        background-color: #1e293b !important;
        border: 1px solid #334155 !important;
        border-radius: 0.5rem !important;
        height: calc(100vh - 300px) !important;
        min-height: 300px !important;
    }
    
    .chat-message {
        background-color: #1e293b !important;
        border: 1px solid #334155 !important;
        border-radius: 0.5rem !important;
        padding: 0.75rem !important;
        margin: 0.5rem 0 !important;
        font-size: 0.95rem !important;
    }
    
    .user-message {
        background-color: #2563eb !important;
    }
    
    /* Form elements */
    textarea, select {
        background-color: #1e293b !important;
        border: 1px solid #334155 !important;
        color: #f1f5f9 !important;
        font-size: 1rem !important;
        padding: 0.75rem !important;
    }
    
    button {
        background-color: #2563eb !important;
        color: white !important;
        padding: 0.75rem 1rem !important;
        font-size: 1rem !important;
        min-height: 44px !important; /* Touch target size */
    }
    
    button:hover {
        background-color: #1d4ed8 !important;
    }
    
    /* Progress sections */
    .tab-nav {
        border-bottom: 1px solid #334155 !important;
    }
    
    .tab-nav button {
        background-color: transparent !important;
        color: #94a3b8 !important;
    }
    
    .tab-nav button.selected {
        color: #f1f5f9 !important;
        border-bottom: 2px solid #2563eb !important;
    }
    
    /* Mobile optimizations */
    @media (max-width: 768px) {
        .gradio-container {
            padding: 0.5rem !important;
        }
        
        .auth-container {
            margin: 1rem !important;
            padding: 1rem !important;
        }
        
        .chat-window {
            height: calc(100vh - 250px) !important;
        }
        
        .chat-message {
            padding: 0.5rem !important;
            font-size: 0.9rem !important;
        }
        
        textarea {
            min-height: 100px !important;
        }
        
        button {
            width: 100% !important;
            margin: 0.25rem 0 !important;
        }
        
        .tab-nav button {
            padding: 0.5rem !important;
            font-size: 0.9rem !important;
        }
    }
    """

This module provides the user interface for the clinical learning system, including:
 * Case presentation and feedback
 * Learning preference configuration
 * Session management
 * Progress visualization

In [ ]:
#| export
class LearningInterface:
    """
    Gradio interface for clinical learning system.
    
    Features:
    - Context selection
    - Goal management
    - Case discussion with streaming
    - Progress tracking
    """
    
    def __init__(
        self,
        context_path: Optional[Path] = None,
        model: str = "anthropic/claude-3.5-sonnet",
        api_url: Optional[str] = None
    ):
        """Initialize interface."""
        default_api_url = "http://170.64.204.93"
        self.tutor = ClinicalTutor(context_path, model)
        self.auth = AuthManager(api_url or default_api_url)

        # Available options
        self.specialties = [
            "Internal Medicine",
            "Emergency Medicine",
            "Surgery",
            "Pediatrics",
            "Family Medicine"
        ]
        
        self.settings = ["Clinic", "Wards", "ED"]
        
        logger.info("Learning interface initialized")

    # Authentication Components
    def _create_auth_components(self) -> Tuple[gr.Group, gr.Group]:
        """Create authentication interface components."""
        with gr.Group() as auth_group:
            gr.Markdown("# Clinical Learning Assistant")
            
            with gr.Tab("Login"):
                login_email = gr.Textbox(
                    label="Email",
                    placeholder="Enter your email"
                )
                login_password = gr.Textbox(
                    label="Password",
                    type="password",
                    placeholder="Enter your password"
                )
                login_button = gr.Button("Login")
                login_message = gr.Markdown()
            
            with gr.Tab("Register"):
                register_email = gr.Textbox(
                    label="Email",
                    placeholder="Enter your email"
                )
                register_password = gr.Textbox(
                    label="Password",
                    type="password",
                    placeholder="Choose a password"
                )
                register_button = gr.Button("Register")
                register_message = gr.Markdown()
        
        # Main interface (hidden until auth)
        main_group = gr.Group(visible=False)
        
        return auth_group, main_group, (
            login_email, login_password, login_button, login_message,
            register_email, register_password, register_button, register_message
        )
    
    def handle_login(
        self,
        email: str,
        password: str,
        state: Dict[str, Any]
    ) -> Tuple[str, gr.update, gr.update, Dict[str, Any]]:
        """Handle user login."""
        success, message = self.auth.login(email, password)
        if success:
            state["authenticated"] = True
            return (
                "Login successful! Redirecting...",  # Success message
                gr.update(visible=False),  # Hide auth group
                gr.update(visible=True),  # Show main interface
                state
            )
        return message, gr.update(), gr.update(), state
    
    def handle_register(self, email: str, password: str) -> str:
        """Handle user registration."""
        success, message = self.auth.register(email, password)
        return message

    # Updated context update to include session tracking
    async def update_context(
        self,
        specialty: str,
        setting: str,
        state: Dict[str, Any]
    ) -> List:
        """Update rotation context with session tracking."""
        if specialty and setting:
            self.tutor.learning_context.update_rotation(specialty, setting)
            
            # Start new session
            if self.auth.start_session(specialty, setting):
                self.auth.log_interaction(
                    "context_update",
                    f"Specialty: {specialty}, Setting: {setting}"
                )
            
            if not state.get("suggested_goals"):
                goals = await self.tutor.generate_smart_goals(specialty, setting)
                state["suggested_goals"] = goals
                
                # Log goal generation
                if self.auth.token:
                    self.auth.log_interaction(
                        "goals_generated",
                        f"Generated {len(goals)} goals"
                    )
            
            return self._update_displays(state)
        return [[], [], [], "No active learning goal"]

    # Chat Processing
    async def process_chat(
        self,
        message: str,
        history: List[Dict[str, str]],
        state: Dict[str, Any]
    ) -> AsyncGenerator[Tuple[List[Dict[str, str]], str, Dict[str, Any]], None]:
        """
        Process chat messages with streaming, history management, and interaction logging.
        
        Args:
            message: User input message
            history: Chat history list of message dictionaries
            state: Current interface state
            
        Yields:
            tuple: (updated history, cleared message input, updated state)
        """
        try:
            if not message.strip():
                yield history, "", state
                return
            
            # Start new discussion if none active
            if not state.get("discussion_active"):
                state["discussion_active"] = True
                state["discussion_start"] = datetime.now().isoformat()
                self.tutor.current_discussion = []  # Reset tutor's discussion history
                # Log discussion start
                self.auth.log_interaction("discussion_start")
            
            # Initialize history if needed
            if history is None:
                history = []
            
            # Add user message
            history.append({
                "role": "user",
                "content": message
            })
            
            # Log user message
            self.auth.log_interaction("user_message", message)
            
            # Add initial assistant message
            history.append({
                "role": "assistant",
                "content": ""
            })
            
            # Update display
            yield history, "", state
            
            # Build messages array for LLM including full history
            messages = [{
                "role": "system",
                "content": self.tutor._build_discussion_prompt()
            }]
            
            # Add previous messages from tutor's discussion history
            messages.extend(self.tutor.current_discussion)
            
            # Add current message
            messages.append({"role": "user", "content": message})
            
            # Process response with streaming
            current_response = ""
            async for token in self.tutor._get_completion_stream(messages):
                current_response += token
                history[-1]["content"] = current_response
                yield history, "", state
            
            # Log assistant response
            self.auth.log_interaction("assistant_response", current_response)
            
            # Update state and discussion history
            state["last_message"] = datetime.now().isoformat()
            
            # Update tutor's discussion history
            self.tutor.current_discussion = history[:-1]  # Don't include empty assistant message
            self.tutor.current_discussion.append({
                "role": "assistant",
                "content": current_response
            })
            
            yield history, "", state
            
        except Exception as e:
            logger.error(f"Error in chat: {str(e)}")
            # Log error
            self.auth.log_interaction("error", str(e))
            
            if history is None:
                history = []
            history.extend([
                {"role": "user", "content": message},
                {"role": "assistant", "content": "I apologize, but I encountered an error. Please try again."}
            ])
            yield history, "", state    
            
    def _update_discussion_status(self, state: Dict[str, Any]) -> str:
        """Update discussion status display."""
        try:
            if not state.get("discussion_active"):
                return "Start a new case discussion"
                
            start = datetime.fromisoformat(state["discussion_start"])
            duration = datetime.now() - start
            minutes = int(duration.total_seconds() / 60)
            
            return f"Active discussion ({minutes} minutes)"
            
        except Exception as e:
            logger.error(f"Error updating status: {str(e)}")
            return "Discussion status unknown"

    async def generate_goals(self, state: Dict[str, Any]) -> List:
        """Generate new goal suggestions."""
        rotation = self.tutor.learning_context.rotation
        if rotation.specialty and rotation.setting:
            goals = await self.tutor.generate_smart_goals(
                rotation.specialty,
                rotation.setting
            )
            state["suggested_goals"] = goals
            return self._update_displays(state)
        return [[], [], [], "No active learning goal"]

    async def add_user_goal(self, text: str, state: Dict[str, Any]) -> List:
        """Add user-specified goal."""
        if not text.strip():
            return self._update_displays(state)
            
        rotation = self.tutor.learning_context.rotation
        if rotation.specialty and rotation.setting:
            goal = await self.tutor.generate_smart_goal(
                text,
                rotation.specialty,
                rotation.setting
            )
            if goal:
                if "suggested_goals" not in state:
                    state["suggested_goals"] = []
                state["suggested_goals"].append(goal)
                return self._update_displays(state)
        return self._update_displays(state)

    def select_goal(self, evt: gr.SelectData, state: Dict[str, Any]) -> List:
        """Set selected goal as active."""
        if "suggested_goals" in state and evt.index[0] < len(state["suggested_goals"]):
            goal = state["suggested_goals"][evt.index[0]]
            self.tutor.learning_context.add_smart_goal(goal)
            return self._update_displays(state)
        return self._update_displays(state)

    def end_discussion(self, state: Dict[str, Any]) -> List:
        """End current discussion with analytics."""
        if state.get("discussion_active"):
            self.auth.log_interaction("discussion_end", json.dumps({
                "duration": (
                    datetime.now() - 
                    datetime.fromisoformat(state["discussion_start"])
                ).total_seconds()
            }))
        self.tutor.end_discussion()
        state["discussion_active"] = False
        state["discussion_start"] = None
        state["last_message"] = None
        self.tutor.current_discussion = []  # Clear discussion history
        return self._update_displays(state)

    def _update_displays(self, state: Dict[str, Any]) -> List:
        """Update all display components."""
        context = self.tutor.learning_context
        
        # Update goals list
        goals_data = []
        for goal in state.get("suggested_goals", []):
            status = "Active" if (
                context.active_goal and 
                context.active_goal.id == goal.id
            ) else "Available"
            
            goals_data.append([
                goal.smart_version,
                goal.category.value,
                status
            ])
        
        # Update progress display
        summary = context.get_category_summary()
        progress_data = [
            [cat, data["completed"], data["total"]]
            for cat, data in summary.items()
        ]
        
        # Update recent completions
        recent_data = []
        for cat, data in summary.items():
            for goal in data["recent"]:
                recent_data.append([
                    goal["smart_version"],
                    cat,
                    goal["completed_at"]
                ])
        
        # Update active goal display
        goal_text = (
            f"Current Goal: {context.active_goal.smart_version}"
            if context.active_goal else
            "No active learning goal"
        )
        
        return [goals_data, progress_data, recent_data, goal_text]

    def create_interface(self) -> gr.Blocks:
        """Create streaming-enabled interface with authentication and all features."""
        with gr.Blocks(title="Clinical Learning Assistant", css=create_css()) as interface:
            # State management
            state = gr.State({
                "discussion_active": False,
                "suggested_goals": [],
                "discussion_start": None,
                "last_message": None,
                "authenticated": False
            })
            
            # Authentication interface
            with gr.Group(elem_classes=["auth-container"]) as auth_group:
                gr.Markdown("# Clinical Learning Assistant")
                
                with gr.Tabs() as auth_tabs:
                    with gr.Tab("Login"):
                        login_email = gr.Textbox(
                            label="Email",
                            placeholder="Enter your email",
                            elem_classes=["auth-input"]
                        )
                        login_password = gr.Textbox(
                            label="Password",
                            type="password",
                            placeholder="Enter your password",
                            elem_classes=["auth-input"]
                        )
                        login_button = gr.Button(
                            "Login",
                            variant="primary",
                            elem_classes=["auth-button"]
                        )
                        login_message = gr.Markdown()
                    
                    with gr.Tab("Register"):
                        register_email = gr.Textbox(
                            label="Email",
                            placeholder="Enter your email",
                            elem_classes=["auth-input"]
                        )
                        register_password = gr.Textbox(
                            label="Password",
                            type="password",
                            placeholder="Choose a password",
                            elem_classes=["auth-input"]
                        )
                        register_button = gr.Button(
                            "Register",
                            variant="primary",
                            elem_classes=["auth-button"]
                        )
                        register_message = gr.Markdown()
            
            # Main interface (initially hidden)
            with gr.Group(visible=False) as main_group:
                with gr.Row():
                    # Context selection
                    specialty = gr.Dropdown(
                        choices=self.specialties,
                        label="Specialty",
                        value=self.tutor.learning_context.rotation.specialty or None
                    )
                    setting = gr.Dropdown(
                        choices=self.settings,
                        label="Setting",
                        value=self.tutor.learning_context.rotation.setting or None
                    )
    
                # Main content
                with gr.Row():
                    # Left: Discussion interface
                    with gr.Column(scale=2):
                        # Active goal display
                        goal_display = gr.Markdown(value="No active learning goal")
    
                        # Discussion status
                        discussion_status = gr.Markdown(
                            value="Start a new case discussion",
                            elem_classes=["discussion-status"]
                        )
    
                        # Chat interface
                        chatbot = gr.Chatbot(
                            height=400,
                            show_label=False,
                            type="messages",  # Use newer message format
                            elem_classes=["chat-window"]
                        )
    
                        with gr.Row():
                            msg = gr.Textbox(
                                label="Present your case or ask questions",
                                placeholder=(
                                    "Present your case as you would to your supervisor:\n"
                                    "- Start with the chief complaint\n"
                                    "- Include relevant history and findings\n"
                                    "- Share your assessment and plan"
                                ),
                                lines=4
                            )
                            audio_input = gr.Audio(
                                sources=["microphone"],
                                type="numpy",
                                label="Or speak your case",
                                streaming=True
                            )
    
                        with gr.Row():
                            clear = gr.Button("Clear Discussion")
                            end = gr.Button(
                                "End Discussion & Review",
                                variant="primary"
                            )
    
                    # Right: Progress & Goals
                    with gr.Column(scale=1):
                        with gr.Tab("Learning Goals"):
                            with gr.Row():
                                generate = gr.Button("Generate New Goals")
                                new_goal = gr.Textbox(
                                    label="Or enter your own goal",
                                    placeholder="What do you want to get better at?"
                                )
                                add_goal = gr.Button("Add Goal")
    
                            goals_list = gr.DataFrame(
                                headers=["Goal", "Category", "Status"],
                                value=[],
                                label="Available Goals",
                                interactive=True,
                                wrap=True
                            )
    
                        with gr.Tab("Progress"):
                            progress_display = gr.DataFrame(
                                headers=["Category", "Completed", "Total"],
                                value=[],
                                label="Progress by Category"
                            )
    
                            recent_display = gr.DataFrame(
                                headers=["Goal", "Category", "Completed"],
                                value=[],
                                label="Recently Completed"
                            )
    
                # Wire up auth events
                login_button.click(
                    self.handle_login,
                    inputs=[login_email, login_password, state],
                    outputs=[login_message, auth_group, main_group, state]
                ).success(
                    lambda: None,  # Clear login form
                    None,
                    [login_email, login_password]
                )
    
                register_button.click(
                    self.handle_register,
                    inputs=[register_email, register_password],
                    outputs=register_message
                )
    
                # Define helper functions
                def clear_discussion() -> Tuple[List, str, Dict]:
                    """Clear chat history."""
                    self.tutor.current_discussion = []  # Clear tutor's discussion history
                    return [], "", {
                        "discussion_active": False,
                        "suggested_goals": [],
                        "discussion_start": None,
                        "last_message": None
                    }
                
                def process_audio(audio: np.ndarray) -> str:
                    """Convert audio to text."""
                    if audio is None:
                        return ""
                    # Add your audio processing logic here
                    return "Audio transcription would appear here"
    
                # Wire up main interface events with tracking
                specialty.change(
                    fn=self.update_context,
                    inputs=[specialty, setting, state],
                    outputs=[goals_list, progress_display, recent_display, goal_display],
                    api_name=False  # Important for async functions
                )
    
                setting.change(
                    fn=self.update_context,
                    inputs=[specialty, setting, state],
                    outputs=[goals_list, progress_display, recent_display, goal_display],
                    api_name=False  # Important for async functions
                )
    
                # Chat events with streaming
                msg.submit(
                    self.process_chat,
                    inputs=[msg, chatbot, state],
                    outputs=[chatbot, msg, state],
                    queue=True  # Important for streaming
                ).then(
                    self._update_discussion_status,
                    inputs=[state],
                    outputs=[discussion_status]
                )
    
                # Voice input handling
                audio_input.stream(
                    process_audio,
                    inputs=[audio_input],
                    outputs=[msg]
                ).then(
                    self.process_chat,
                    inputs=[msg, chatbot, state],
                    outputs=[chatbot, msg, state]
                )
    
                # Button handlers
                clear.click(
                    clear_discussion,
                    outputs=[chatbot, msg, state]
                ).then(
                    lambda: "Start a new case discussion",
                    outputs=[discussion_status]
                )
    
                end.click(
                    self.end_discussion,
                    inputs=[state],
                    outputs=[goals_list, progress_display, recent_display, goal_display]
                )
    
                generate.click(
                    self.generate_goals,
                    inputs=[state],
                    outputs=[goals_list, progress_display, recent_display, goal_display]
                )
    
                # Goal management
                add_goal.click(
                    self.add_user_goal,
                    inputs=[new_goal, state],
                    outputs=[goals_list, progress_display, recent_display, goal_display]
                )
    
                goals_list.select(
                    self.select_goal,
                    inputs=[state],
                    outputs=[goals_list, progress_display, recent_display, goal_display]
                )
    
            return interface

## Tests

In [ ]:
#| hide
async def test_learning_interface():
    """Test learning interface functionality"""
    interface = LearningInterface()
    
    # Test chat processing
    history = []
    test_input = "28yo M with chest pain"
    
    new_history, msg = await interface.process_chat(test_input, history)
    assert isinstance(new_history, list)
    assert len(new_history) == 2  # User message + response
    assert new_history[0]["role"] == "user"
    assert new_history[0]["content"] == test_input
    
    # Test discussion analysis
    analysis = await interface.end_discussion(new_history)
    assert isinstance(analysis, dict)
    assert all(k in analysis for k in [
        'learning_points', 'gaps', 'strengths', 'suggested_objectives'
    ])
    
    # Test rotation updates
        rotation = interface.update_rotation(
        "Emergency Medicine",
        "2025-01-01",
        "2025-03-31",
        ["Resuscitation", "Procedures"]
    )
    assert rotation["specialty"] == "Emergency Medicine"
    assert "Resuscitation" in rotation["key_focus_areas"]
    
    # Test objective management
    objectives = interface.toggle_objective("Improve chest pain assessment", False)
    assert len(objectives) == 1
    assert objectives[0]["status"] == "active"
    
    objectives = interface.toggle_objective("Improve chest pain assessment", True)
    assert objectives[0]["status"] == "completed"
    
    # Test feedback preferences
    preferences = interface.toggle_feedback("Include more ddx", True)
    assert len(preferences) == 1
    assert preferences[0]["active"] == True
    
    print("Interface tests passed!")

# Run tests
if __name__ == "__main__":
    import asyncio
    if not asyncio.get_event_loop().is_running():
        asyncio.run(test_learning_interface())